In [1]:
import pandas as pd
import json
import requests
import datetime as datetime
from datetime import date
from dateutil.parser import *

In [2]:
url = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Use_of_Force/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

In [3]:
response = requests.get(url)

In [4]:
response_json = response.json()

In [5]:
response_json.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'features'])

In [7]:
len(response_json["features"])

33753

In [8]:
response_json["features"][0]['attributes'].keys()

dict_keys(['PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem', 'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber', 'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction', 'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct', 'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear', 'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude', 'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'])

In [9]:
new_dict = {}
new_dict["Year"] = []
new_dict["Year-Month"] = []
new_dict["Hour"] = []
for field in response_json["features"][0]['attributes'].keys():
    new_dict[field] = []
for index in range(len(response_json["features"])):
    a = response_json["features"][index]['attributes']
    for key in response_json["features"][index]['attributes']:
#         print(key, a[key])
        if key == 'ResponseDate':
            d = datetime.datetime.fromtimestamp(a[key]/1000)
            e = d.strftime('%Y-%m-%d')
            # e = d.strftime('%Y-%m-%d %H:%M:%S')
            new_dict[key].append(e)
            new_dict["Year"].append(d.strftime('%Y'))
            new_dict["Year-Month"].append(d.strftime('%Y-%m'))
            new_dict["Hour"].append(d.strftime('%H'))
        else:
            new_dict[key].append(a[key])
    if index > 10000:
        break
df = pd.DataFrame(new_dict)

In [8]:
df.head()

,PoliceUseOfForceID,Year,Year-Month,Hour,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,...,TotalCityCallsForYear,TotalPrecinctCallsForYear,TotalNeighborhoodCallsForYear,CenterGBSID,CenterLatitude,CenterLongitude,CenterX,CenterY,DateAdded,OBJECTID
0,24543755,2007,2007-12,19,08-000149,2007-12-31,Code 3,No,MISC,None,...,322402,46998,23458,17121,44.980808,-93.273141,-1.038312e+07,5.618501e+06,1626941998000,1
1,24543756,2008,2008-01,21,08-002346,2008-01-02,Theft,Yes,BURGTL,No,...,322402,80434,1123,16516,44.988830,-93.291040,-1.038511e+07,5.619763e+06,1626941998000,2
2,24543757,2008,2008-01,21,08-014491,2008-01-14,Check the Welfare,No,CIC,No,...,322402,80434,2992,10361,45.022277,-93.310208,-1.038724e+07,5.625029e+06,1626941998000,3
3,24543758,2008,2008-02,18,08-038762,2008-02-08,Recover Vehicle,No,RECVEH,No,...,322402,80434,7483,10416,45.016752,-93.296221,-1.038569e+07,5.624159e+06,1626941998000,4
4,24543759,2008,2008-02,10,08-054881,2008-02-26,Suspicious Person,No,OBSTRU,No,...,322402,84018,10316,20569,44.963324,-93.265106,-1.038222e+07,5.615749e+06,1626941998000,5


In [17]:
new_dict.keys()

dict_keys(['Year', 'Year-Month', 'Hour', 'PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem', 'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber', 'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction', 'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct', 'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear', 'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude', 'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'])

In [ ]:
db_Table_Create = {}
for key in new_dict.keys():
    entry = f"{key} TEXT"
    

In [39]:
# Table Creation String
db_Table_Create = [f"{key} TEXT" for key in new_dict.keys()]
Create_Table_String = ",".join(db_Table_Create)
Create_Table_String

'Year TEXT,Year-Month TEXT,Hour TEXT,PoliceUseOfForceID TEXT,CaseNumber TEXT,ResponseDate TEXT,Problem TEXT,Is911Call TEXT,PrimaryOffense TEXT,SubjectInjury TEXT,ForceReportNumber TEXT,SubjectRole TEXT,SubjectRoleNumber TEXT,ForceType TEXT,ForceTypeAction TEXT,Race TEXT,Sex TEXT,EventAge TEXT,TypeOfResistance TEXT,Precinct TEXT,Neighborhood TEXT,TotalCityCallsForYear TEXT,TotalPrecinctCallsForYear TEXT,TotalNeighborhoodCallsForYear TEXT,CenterGBSID TEXT,CenterLatitude TEXT,CenterLongitude TEXT,CenterX TEXT,CenterY TEXT,DateAdded TEXT,OBJECTID TEXT'

In [33]:
# Populate Database
x=0
for x in range(3):
    print([new_dict[key][x] for key in new_dict.keys()])

['2007', '2007-12', '19', 24543755, '08-000149', '2007-12-31', 'Code 3 ', 'No', 'MISC', None, 1, 'OT', 1, 'Bodily Force', 'Body Weight to Pin', 'White', 'Male', 39, 'Commission of Crime', '01', 'Downtown West', 322402, 46998, 23458, 17121, 44.980808, -93.273141, -10383118.5232, 5618500.6487, 1626941998000, 1]
['2008', '2008-01', '21', 24543756, '08-002346', '2008-01-02', 'Theft ', 'Yes', 'BURGTL', 'No', 3, 'A', 1, 'Bodily Force', 'Knees', 'Other / Mixed Race', 'Male', 15, 'Fled on Foot', '04', 'Sumner - Glenwood', 322402, 80434, 1123, 16516, 44.98883, -93.29104, -10385111.0281, 5619763.1442, 1626941998000, 2]
['2008', '2008-01', '21', 24543757, '08-014491', '2008-01-14', 'Check the Welfare ', 'No', 'CIC', 'No', 1, 'OT', 1, 'Taser', 'Firing Darts', 'White', 'Male', 29, 'Verbal Non-Compliance', '04', 'Cleveland', 322402, 80434, 2992, 10361, 45.022277, -93.310208, -10387244.8903, 5625029.1676, 1626941998000, 3]


In [22]:
response_json["features"][0]['geometry']

{'x': -93.2969048626618, 'y': 44.92590912407578}

In [32]:
b = response_json["features"][0]['attributes']['ResponseDate']

In [ ]:
# Map Data Table - CaseNumber, Lat, Long, Neighborhood, Precinct
# Incident Data Table - everything else

In [11]:
import sqlite3

In [14]:
connection = sqlite3.connect('MNPDUseofForce.db')

In [15]:
cursor = connection.cursor()

In [16]:
cursor.execute('''CREATE TABLE IF NOT EXISTS Shows
              (Title TEXT, Director TEXT, Year INT)''')

connection.commit()
connection.close()

In [ ]:
# cursor.execute("INSERT INTO Shows VALUES ('Stranger Things', 'Shawn Levy', 2016)")